In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/Projects/github/Major_ML_models_implemented_in_PyTorch/GPT

/content/drive/My Drive/Projects/github/Major_ML_models_implemented_in_PyTorch/GPT


## Train a character-level GPT 


In [4]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [7]:
# make deterministic
from utils import set_seed
set_seed(42)

In [8]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [9]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - (self.block_size + 1)

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [12]:
block_size = 128 # spatial extent of the model for its context
text = open('input.txt', 'r').read() 
train_dataset = CharDataset(text, block_size) 

data has 1155775 characters, 76 unique.


In [14]:
from config import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

08/26/2020 04:27:42 - INFO - model -   number of parameters: 2.536346e+07


In [16]:
import gc
gc.collect()

0

In [ ]:
from train import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=2, batch_size=256, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

In [20]:
# alright, let's sample some character-level Shakespeare
from utils import sample

context = "O God, O God!"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

O God, O God! that e'er this tongue of mine,\
That laid the sentence of dread banishment\
On yon proud man, should take it off again\
With words of sooth! O that I were as great\
As is my grief, or lesser than my name!\
Or that I could forget what I have been,\
Or not remember what I must be now!\
Swell'st thou, proud heart? I'll give thee scope to beat,\
Since foes have scope to beat both thee and me.\
\
DUKE OF AUMERLE:\
Northumberland comes back from Bolingbroke.\
\
KING RICHARD II:\
What must the king do now? must he submit?\
The king shall do it: must he be deposed?\
The king shall be contented: must he lose\
The name of king? o' God's name, let it go:\
I'll give my jewels for a set of beads,\
My gorgeous palace for a hermitage,\
My gay apparel for an almsman's gown,\
My figured goblets for a dish of wood,\
My sceptre for a palmer's walking staff,\
My subjects for a pair of carved saints\
And my large kingdom for a little grave,\
A little little grave, an obscure grave;\
Or I'll b